In [ ]:
import numpy as np 
import pandas as pd
import os
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from scipy import stats
import seaborn as sns; sns.set()
import warnings
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.utils import shuffle
warnings.filterwarnings(action = 'ignore') 
import gensim 
from gensim.models import Word2Vec
import sys
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedBaggingClassifier

In [ ]:
data=pd.read_csv('/content/train_DPC_AAC_PCP.csv')
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [ ]:
#Forest = RandomForestClassifier(random_state = 42)
#n_estimators = [200, 400, 600, 800, 1000, 1200]
#max_depth = [5, 8, 15, 25, 30]
#min_samples_split = [2, 5, 10, 15, 100]
#min_samples_leaf = [1, 2, 5, 10] 

#hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
#              min_samples_split = min_samples_split, 
#             min_samples_leaf = min_samples_leaf)

#gridF = GridSearchCV(Forest, hyperF, cv = 3, verbose = 1, 
#                      n_jobs = -1)
#bestF = gridF.fit(X_train, y_train)

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(clf, param_distributions=params, n_iter=param_comb,
                                   scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3,
                                   random_state=1001 )
# Here we go
#start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)

NameError: ignored

In [ ]:
#FOR AUC
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=10, random_state=42, shuffle=True)
kf.get_n_splits(X)
cc = []
dd = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf = RandomForestClassifier(random_state = 42, n_estimators = 100)
    clf.fit(X_train,y_train)
    cc.append(clf.predict_proba(X_test))
    dd.append(y_test)
    flat_pred = [x for sublist in cc for x in sublist]
    flat_true = [x for sublist in dd for x in sublist]

KeyboardInterrupt: ignored

In [ ]:
#FOR ACCURACY ETC
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=10, random_state=42, shuffle=True)
kf.get_n_splits(X)
cc = []
dd = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Training
    clf = BalancedBaggingClassifier(base_estimator = SVC(random_state = 123),
                                    n_estimators = 100, random_state = 123, verbose = 1, n_jobs = -1)
    clf.fit(X_train, y_train)
    cc.append(clf.predict(X_test))
    dd.append(y_test)
    flat_pred = [x for sublist in cc for x in sublist]
    flat_true = [x for sublist in dd for x in sublist]
print('Confusion Matrix:')
conf_matx= confusion_matrix(flat_true,flat_pred)
print('Recall:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[1][0]))
print('Sensitivity:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[0][1]))
print('Specificity:', conf_matx[1][1]/(conf_matx[1][1]+conf_matx[0][1]))
print('Accuracy:', (conf_matx[0][0]+conf_matx[1][1])/(conf_matx[1][1]+conf_matx[0][1]+conf_matx[1][0]+conf_matx[0][0]))

#Testing
data2=pd.read_csv('/content/test_DPC_AAC_PCP.csv')
z_pr = data2.iloc[:,:-1]
z_test = data2.iloc[:,-1]
z_pred=clf.predict(z_pr)
# print("Accuracy:",metrics.accuracy_score(z_test, z_pred))
aa = []
for i in range(0,len(z_test)):
  if z_test[i] == 'Y':
    aa.append(1)
  else:
    aa.append(0)

conf_matx= confusion_matrix(z_test,z_pred)
print('Recall:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[1][0]))
print('Sensitivity:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[0][1]))
print('Specificity:', conf_matx[1][1]/(conf_matx[1][1]+conf_matx[0][1]))
print('Accuracy:', (conf_matx[0][0]+conf_matx[1][1])/(conf_matx[1][1]+conf_matx[0][1]+conf_matx[1][0]+conf_matx[0][0]))

z_pred_1=clf.predict_proba(z_pr)
df123 = pd.DataFrame(z_pred_1)

#Testing#
from sklearn.metrics import roc_auc_score
roc_auc_score(z_test, clf.predict_proba(z_pr)[:, 1])

#Training#
flat_pred_1 = pd.DataFrame(flat_pred)
from sklearn.metrics import roc_auc_score
roc_auc_score(flat_true,flat_pred_1.iloc[:,-1].ravel())

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 16.9min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 16.9min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.7min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 17.0min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 17.0min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.9min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.9min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Don

Confusion Matrix:
Recall: 0.6761188930645712
Sensitivity: 0.668355285376562
Specificity: 0.6754791804362195
Accuracy: 0.6757937174533849


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 16.7min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 16.7min finished


Recall: 0.7571027162035592
Sensitivity: 0.7043276212605286
Specificity: 0.6063418406805878
Accuracy: 0.6897564346173778


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 16.7min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 16.7min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 16.4min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 16.4min finished


0.6757551827952328

In [ ]:
def Model_1(train, test):
    ''' Trains the model and Saves the predictions in a CSV file
        train : Training set
        test : Test set
    '''
# Preprocessing
    X_train = OneHotEncoder(sparse = False).fit_transform([[x for x in s] for s in X])
    X_test = OneHotEncoder(sparse = False).fit_transform([[x for x in s] for s in y])
    Y_train = train['label']

    X_train, Y_train = RandomUnderSampler(random_state = 100).fit_resample(X_train, Y_train)
    X_train, Y_train = shuffle(X_train, Y_train, random_state = 100)

    # Training
    estimators = [
                ('rf', RandomForestClassifier(n_estimators = 300, max_depth = 45, min_samples_leaf = 7, random_state = 100)),
                ('mlp', MLPClassifier(max_iter = 200,random_state = 100)),
                ('knn', KNeighborsClassifier(n_neighbors = 4))]

    clf = StackingClassifier(estimators = estimators, final_estimator=LogisticRegression(random_state = 100),
                             n_jobs = -1, verbose = 1)
    clf.fit(X_train, Y_train)

    # Predicting
    Y_pred = clf.predict(X_test)
    Y_prob = [x[1] for x in clf.predict_proba(X_test)]
    result = pd.DataFrame()
    result["ID"] = test["ID"]
    result["Label"] = Y_prob
    result.to_csv("Submission_1.csv", index = False)
    result["Label"] = Y_pred
    result.to_csv("Predictions_1.csv", index = False)


In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
data2=pd.read_csv('/content/test_DPC.csv')
z_pr = data2.iloc[:,:-1]
z_test = data2.iloc[:,-1]
z_pred=clf.predict(z_pr)
# print("Accuracy:",metrics.accuracy_score(z_test, z_pred))
aa = []
for i in range(0,len(z_test)):
  if z_test[i] == 'Y':
    aa.append(1)
  else:
    aa.append(0)

In [ ]:
conf_matx= confusion_matrix(z_test,z_pred)
print('Recall:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[1][0]))
print('Sensitivity:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[0][1]))
print('Specificity:', conf_matx[1][1]/(conf_matx[1][1]+conf_matx[0][1]))
print('Accuracy:', (conf_matx[0][0]+conf_matx[1][1])/(conf_matx[1][1]+conf_matx[0][1]+conf_matx[1][0]+conf_matx[0][0]))

Recall: 0.8238580348340454
Sensitivity: 0.7281440604124311
Specificity: 0.6591405680990532
Accuracy: 0.7457246501986526


In [ ]:
z_pred_1=clf.predict_proba(z_pr)

In [ ]:
df123 = pd.DataFrame(z_pred_1)

In [ ]:
#Testing#
from sklearn.metrics import roc_auc_score
roc_auc_score(z_test, clf.predict_proba(z_pr)[:, 1])

In [ ]:
#Training#
flat_pred_1 = pd.DataFrame(flat_pred)
from sklearn.metrics import roc_auc_score
roc_auc_score(flat_true,flat_pred_1.iloc[:,-1].ravel())

0.6911994972070066